In [1]:
import numpy as np
import whisper
import onnxruntime as ort

tokenizer = whisper.tokenizer.get_tokenizer(multilingual=False)

enc = ort.InferenceSession("encoder.onnx")
dec = ort.InferenceSession("decoder.onnx")

audio = whisper.load_audio("sample.mp3")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).unsqueeze(0).numpy().astype(np.float32)

audio_features = enc.run(None, {"mel": mel})[0]

# CRITICAL FIX: match export shape, then slice to empty
cache = np.zeros((8,1,1,384), dtype=np.float32)
cache = cache[:, :, :0, :]   # now cache length = 0

tokens = np.array([[tokenizer.sot, tokenizer.no_timestamps]], dtype=np.int64)

logits, cache = dec.run(None, {
    "tokens": tokens,
    "audio": audio_features,
    "cache": cache,
})

result = tokens[0].tolist()

for _ in range(200):

    next_token = np.argmax(logits[:, -1, :], axis=-1).reshape(1,1).astype(np.int64)

    if next_token[0,0] == tokenizer.eot:
        break

    result.append(int(next_token[0,0]))

    logits, cache = dec.run(None, {
        "tokens": next_token,
        "audio": audio_features,
        "cache": cache,
    })

print(tokenizer.decode(result))


<|startoftranscript|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|notimestamps|><|n